In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [7]:
corneal = pd.read_csv('corneal.csv')
irritation = pd.read_csv('irritation.csv')
melanin = pd.read_csv('melanin.csv')
#cox2 = pd.read_pdb('COX-2.pdb')
display(corneal)
display(irritation)
display(melanin)

,SMILES,logPerm
0,CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)...,5.135798
1,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O,5.347108
2,CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)N,5.393628
3,C(C(CO)O)O,3.806662
4,CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O,4.442651
...,...,...
115,C1CN2C(=CC=C2C(=O)C3=CC=CC=C3)C1C(=O)O,1.249902
116,CC1CN(CCC1(C2=CC=CC=C2)C(=O)O)C3CCC(CC3)(C#N)C...,1.406097
117,COC1=CC2=C(C=CN=C2C=C1)C(C3CC4CCN3CC4C=C)O,2.161022
118,C1CN=C(N1)NC2=C(C=CC3=NSN=C32)Cl,3.850998


,SMILES,Class
0,CC#CC,1
1,CCC=C,1
2,O=CC=O,1
3,CCC=O,1
4,FCC#N,1
...,...,...
5215,CC1OC(CC(O)C1O)OC2C(O)CC(OC3C(O)CC(OC4CCC5(C)C...,0
5216,CC1OC(CC(O)C1O)OC2C(O)CC(OC3C(O)CC(OC4CCC5(C)C...,0
5217,CN(C(=O)C)c1c(I)c(NC(=O)C)c(I)c(C(=O)NC(C=O)C(...,0
5218,CC(=O)N(CC(O)CO)c1c(I)c(C(=O)NCCO)c(I)c(C(=O)N...,0


,SMILES,Class
0,CCN(CC)CCNC(=O)c1ccc(cc1)N.Cl,1
1,COCCNC(=O)CN1C2CCC1CC(C2)(c3cccnc3)O,1
2,CC1=NN=C(c2cc3c(cc2C1)OCO3)c4ccc(cc4)N,1
3,CC1C2Cc3ccc(cc3C1(CCN2CC=C)C)O,1
4,COc1ccc(cc1)c2coc3cc(ccc3c2=O)OC,1
...,...,...
775,c1ccc(cc1)c2cc(nc(n2)N)c3[nH]c4c(n3)cccn4,0
776,COc1ccc(cc1S(=O)(=O)N2CCc3c2cc(cc3)C(=O)Nc4nc(...,1
777,Cn1ccnc1c2c(cnc(n2)NCCOC)c3cccc(c3)OC,1
778,Cc1cnc(s1)Nc2c(nccn2)C3CCN(CC3)C,0


In [9]:
corneal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SMILES   120 non-null    object 
 1   logPerm  120 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.0+ KB


In [15]:
X = corneal.iloc[:, :-1]
y = corneal.iloc[:, -1]
print(X)
print(y)

                                                SMILES
0    CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)...
1                CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O
2                CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)N
3                                           C(C(CO)O)O
4      CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O
..                                                 ...
115             C1CN2C(=CC=C2C(=O)C3=CC=CC=C3)C1C(=O)O
116  CC1CN(CCC1(C2=CC=CC=C2)C(=O)O)C3CCC(CC3)(C#N)C...
117         COC1=CC2=C(C=CN=C2C=C1)C(C3CC4CCN3CC4C=C)O
118                   C1CN=C(N1)NC2=C(C=CC3=NSN=C32)Cl
119  CC(C1=NC=NC=C1F)C(CN2C=NC=N2)(C3=C(C=C(C=C3)F)F)O

[120 rows x 1 columns]
0      5.135798
1      5.347108
2      5.393628
3      3.806662
4      4.442651
         ...   
115    1.249902
116    1.406097
117    2.161022
118    3.850998
119    3.455686
Name: logPerm, Length: 120, dtype: float64


In [17]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
    }
    return descriptors

# Применение функции к данным
corneal['descriptors'] = corneal['SMILES'].apply(smiles_to_descriptors)
descriptors_df = pd.json_normalize(corneal['descriptors'])
data = pd.concat([corneal, descriptors_df], axis=1)

In [18]:
from sklearn.model_selection import train_test_split

X = data.drop(['SMILES', 'logPerm', 'descriptors'], axis=1)
y = data['logPerm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score



model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, R2: {r2}')

MSE: 1.4342984984816705, R2: 0.4302994420749725


In [20]:
importances = model.feature_importances_
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print(feature_importance)

             feature  importance
1               LogP    0.594256
0              MolWt    0.197352
4  NumRotatableBonds    0.085039
3      NumHAcceptors    0.079561
2         NumHDonors    0.043792


In [21]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Загрузка данных
data = pd.read_csv('corneal.csv')

# Извлечение дескрипторов
def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
    }
    return descriptors

data['descriptors'] = data['SMILES'].apply(smiles_to_descriptors)
descriptors_df = pd.json_normalize(data['descriptors'])
data = pd.concat([data, descriptors_df], axis=1)

# Подготовка данных
X = data.drop(['SMILES', 'logPerm', 'descriptors'], axis=1)
y = data['logPerm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, R2: {r2}')

# Важность признаков
importances = model.feature_importances_
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print(feature_importance)

MSE: 1.4342984984816705, R2: 0.4302994420749725
             feature  importance
1               LogP    0.594256
0              MolWt    0.197352
4  NumRotatableBonds    0.085039
3      NumHAcceptors    0.079561
2         NumHDonors    0.043792


In [30]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Параметры для поиска
param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Модель
rf = RandomForestRegressor(random_state=42)

# Поиск
search = RandomizedSearchCV(
    rf, 
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='r2',
    random_state=42,
    n_jobs=-1
)
search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", search.best_params_)
print("Лучший R² на кросс-валидации:", search.best_score_)

# Оценка на тестовых данных
best_rf = search.best_estimator_
y_pred = best_rf.predict(X_test)
print("Тестовый R²:", r2_score(y_test, y_pred))

Лучшие параметры: {'max_depth': 20, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 409}
Лучший R² на кросс-валидации: 0.21507664557265277
Тестовый R²: 0.5364757420623669


In [28]:
from mordred import Calculator, descriptors

# Инициализация калькулятора
calc = Calculator(descriptors, ignore_3D=True)
mol_list = [Chem.MolFromSmiles(smi) for smi in data['SMILES']]

# Вычисление дескрипторов
mordred_features = calc.pandas(mol_list)
mordred_features = mordred_features.astype(float)

# Объединение с исходными данными
data_extended = pd.concat([data, mordred_features], axis=1)

  7%|▋         | 8/120 [00:23<06:33,  3.51s/it]

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 89%|████████▉ | 107/120 [00:24<00:01, 11.64it/s]

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 120/120 [00:25<00:00,  4.78it/s]


In [29]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Параметры для поиска
params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_samples_leaf': [1, 2]
}

# Поиск
gb = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(gb, params, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Результаты
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший R²:", grid_search.best_score_)

Лучшие параметры: {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'n_estimators': 200}
Лучший R²: 0.23626716123140126


In [25]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Оставляем 95% дисперсии
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Обучение модели на PCA-признаках
model_pca = RandomForestRegressor()
model_pca.fit(X_train_pca, y_train)
print("R² с PCA:", model_pca.score(X_test_pca, y_test))

R² с PCA: -0.4832756631775015


In [26]:
{'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300, 'max_features': 'sqrt'}

{'max_depth': 20,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 300,
 'max_features': 'sqrt'}

In [27]:
# После подбора параметров и добавления дескрипторов
best_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    min_samples_leaf=2,
    random_state=42
)
best_model.fit(X_train_extended, y_train)
y_pred = best_model.predict(X_test_extended)
print("Итоговый R²:", r2_score(y_test, y_pred))  # Например, 0.68

NameError: name 'X_train_extended' is not defined

In [31]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Загрузка данных
data = pd.read_csv('corneal.csv')

# Функция для извлечения дескрипторов из SMILES
def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
    }
    return descriptors

# Добавление дескрипторов в данные
data['descriptors'] = data['SMILES'].apply(smiles_to_descriptors)
descriptors_df = pd.json_normalize(data['descriptors'])
data = pd.concat([data, descriptors_df], axis=1)

# Подготовка данных
X = data.drop(['SMILES', 'logPerm', 'descriptors'], axis=1)
y = data['logPerm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение сетки гиперпараметров
param_grid = {
    'n_estimators': [100, 200, 300],  # Количество деревьев
    'max_depth': [None, 10, 20, 30],   # Максимальная глубина
    'min_samples_split': [2, 5, 10],    # Минимальное число образцов для разделения
    'min_samples_leaf': [1, 2, 4],      # Минимальное число образцов в листе
    'max_features': ['sqrt', 'log2']    # Количество признаков для разделения
}

# Инициализация модели
rf = RandomForestRegressor(random_state=42)

# Поиск по сетке с кросс-валидацией (cv=5)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,  # Использование всех ядер CPU
    verbose=2
)

# Запуск поиска
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Оценка лучшей модели
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R² на тестовых данных: {r2:.4f}")
print(f"MSE на тестовых данных: {mse:.4f}")

# Важность признаков
importances = best_rf.feature_importances_
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nВажность признаков:")
print(feature_importance.head(10))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Лучшие параметры: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
R² на тестовых данных: 0.5856
MSE на тестовых данных: 1.0434

Важность признаков:
             feature  importance
1               LogP    0.472088
3      NumHAcceptors    0.156487
0              MolWt    0.153484
4  NumRotatableBonds    0.110872
2         NumHDonors    0.107069
